<a href="https://colab.research.google.com/github/limweiliang/weed-classifier/blob/main/CS3244_ProjectCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [2]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sklearn
import math

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Define constants
RANDOM_STATE = 4242
NUM_CLASSES = 9

# ds = tfds.load('deep_weeds', batch_size = -1, as_supervised= True)
# images, labels = ds['train'] # Type: EagerTensor

# # Shuffle the dataset
# images = tf.random.shuffle(images, seed=RANDOM_STATE)
# labels = tf.random.shuffle(labels, seed=RANDOM_STATE)

# print(images.shape, labels.shape)

# # Split dataset into train-val-test
# num_images = images.shape[0]
# last_train_image = math.floor(num_images * 0.6)
# last_val_image = math.floor(num_images * 0.8)

# X_train = images[:last_train_image]
# y_train = labels[:last_train_image]
# X_val = images[last_train_image:last_val_image]
# y_val = labels[last_train_image:last_val_image]
# X_test = images[last_val_image:]
# y_test = labels[last_val_image:]

# print(X_train.shape, X_val.shape, X_test.shape)

In [5]:
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

train_ds, validation_ds, test_ds = tfds.load('deep_weeds', as_supervised= True, shuffle_files= True, split=["train[:70%]", "train[70%:90%]", "train[90%:100%]"], batch_size = BATCH_SIZE)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print("Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds))
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

def one_hot(img, label):
    return (img, tf.one_hot(label,9))

def add_grey_grad(img ,label):
    org_img = img
    img = tf.cast(img, float)
    #img = tf.expand_dims(img, axis = 0)
    gray = tf.image.rgb_to_grayscale(img)
    dy , dx = tf.image.image_gradients(gray)
    dmag = tf.math.sqrt(tf.add(tf.math.square(dy),tf.math.square(dx)))
    mean = tf.math.reduce_sum(dmag)/(256*256)
    out = (dmag / mean) * 255
    out = tf.expand_dims(tf.cast(out, tf.uint8), axis = 0)
    gray = tf.expand_dims(tf.cast(gray, tf.uint8), axis = 0)
    org_img = tf.concat([org_img,gray[0],out[0]],-1)
    return (org_img, label)

train_ds = train_ds.map(one_hot)
validation_ds = validation_ds.map(one_hot)
test_ds = test_ds.map(one_hot)

train_ds = train_ds.map(add_grey_grad)
validation_ds = validation_ds.map(add_grey_grad)
test_ds = test_ds.map(add_grey_grad)


train_ds = train_ds.shuffle(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE).cache()
validation_ds = validation_ds.shuffle(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE).cache()
test_ds = test_ds.shuffle(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE).cache()

Number of training samples: 192
Number of validation samples: 55
Number of test samples: 28


In [6]:
# Clean up memory usage
# del ds
# del images
# del labels

In [7]:
from tensorflow.keras import datasets, layers, models, regularizers, Input
import datetime

In [8]:
inputs = Input(shape = (256,256,5), name = "Original_Image")
flip = layers.RandomFlip("horizontal_and_vertical")(inputs)
rotate = layers.RandomRotation((-0.5,0.5), fill_mode = "nearest")(flip)
rescale = layers.Rescaling(1/255)(rotate)

x = layers.Conv2D(64, (3,3), padding = "same", strides = 2)(rescale)
x = layers.Conv2D(96, (3,3), padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
block_1_output = layers.MaxPool2D()(x)

x = layers.Conv2D(96, (3,3), padding = "same", strides = 2)(block_1_output)
x = layers.Conv2D(64, (3,3), padding = "same", strides = 2)(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
block_2_output = layers.MaxPool2D()(x)

x = layers.Conv2D(64, (3,3), padding = "same", strides = 2)(block_2_output)
x = layers.Conv2D(64, (3,3), padding = "same", strides = 2)(x)
x = layers.BatchNormalization()(x)
block_3_output = layers.ReLU()(x)

x = layers.Flatten()(block_3_output)
x = layers.Dense(128)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(9, activation = 'softmax')(x)

model = tf.keras.Model(inputs, outputs, name="3_block_CNN")
model.summary()

Model: "3_block_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Original_Image (InputLayer)  [(None, 256, 256, 5)]     0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 256, 256, 5)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 256, 256, 5)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 256, 256, 5)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 64)      2944      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 96)      55392     
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 96)      

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.01)


model.compile(optimizer=optimizer,
              loss= 'categorical_crossentropy',
              metrics=['accuracy', "Precision", "Recall"])

In [10]:
model.summary()

Model: "3_block_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Original_Image (InputLayer)  [(None, 256, 256, 5)]     0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 256, 256, 5)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 256, 256, 5)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 256, 256, 5)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 64)      2944      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 96)      55392     
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 96)      

In [11]:
EarlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True , verbose = 1)

#Constant learning rate for first N epochs then it decreases exponentially
def scheduler(epoch, lr):
  if epoch < 65:
    return lr
  else:
    return lr * tf.math.exp(-0.1/5)

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001, cooldown=10, verbose = 1)

In [ ]:
hist = model.fit(train_ds, epochs=200,
                     validation_data= validation_ds, callbacks=[EarlyStop, reduce_lr],batch_size = BATCH_SIZE)

Epoch 1/200
192/192 [==============================] - 55s 231ms/step - loss: 1.6258 - accuracy: 0.5031 - precision: 0.6522 - recall: 0.3272 - val_loss: 1.3905 - val_accuracy: 0.5357 - val_precision: 0.8144 - val_recall: 0.2707
Epoch 2/200
192/192 [==============================] - 41s 213ms/step - loss: 1.3003 - accuracy: 0.5470 - precision: 0.7380 - recall: 0.3867 - val_loss: 1.4241 - val_accuracy: 0.5214 - val_precision: 0.7312 - val_recall: 0.3107
Epoch 3/200
192/192 [==============================] - 42s 217ms/step - loss: 1.2189 - accuracy: 0.5659 - precision: 0.7587 - recall: 0.4071 - val_loss: 1.4593 - val_accuracy: 0.5220 - val_precision: 0.7689 - val_recall: 0.1710
Epoch 4/200
192/192 [==============================] - 42s 216ms/step - loss: 1.1625 - accuracy: 0.5842 - precision: 0.7621 - recall: 0.4305 - val_loss: 2.6279 - val_accuracy: 0.2873 - val_precision: 0.4302 - val_recall: 0.1788
Epoch 5/200
192/192 [==============================] - 43s 223ms/step - loss: 1.1334 - a

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
precision = hist.history['precision']
val_precision = hist.history['val_precision']
recall = hist.history['recall']
val_recall = hist.history['val_recall']

# Plot the graph manually
epochs = range(len(loss))

plt.figure(figsize=(25, 25))
plt.subplot(4, 1, 1)
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.subplot(4, 1, 2)
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(4, 1, 3)
plt.plot(epochs, precision, 'r', label='Training Precision')
plt.plot(epochs, val_precision, 'b', label='Validation Precision')
plt.title('Training and validation Precision')
plt.legend()

plt.subplot(4, 1, 4)
plt.plot(epochs, recall, 'r', label = "Training Recall")
plt.plot(epochs, val_recall, 'b', label='Validation Recall')
plt.title('Training and validation Recall')
plt.legend()

plt.show()

In [ ]:
model.evaluate(validation_ds)

In [ ]:
model.save('./CNN_Models/3_block_cnn/3_blockV2.h5')

In [ ]:
test = tfds.as_numpy(test_ds)
y_pred = []
y_label = []

for i in test:
  flat_img = i[0]
  flat_label = i[1]
  pred = model.predict(flat_img)
  pred = np.argmax(pred, axis=1)
  y_pred.extend(pred)
  y_label.extend(np.argmax(flat_label, axis = 1))

y_pred = np.array(y_pred)
y_label = np.array(y_label)

In [ ]:
len(y_pred)

In [ ]:
from sklearn.metrics import classification_report

print(y_pred)
print(classification_report(y_label, y_pred))